In [24]:
# Example： 奇摩電影
import requests
from bs4 import BeautifulSoup

url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
r = requests.get(url)
soup = BeautifulSoup(r.text)

infos = []
imgs = []
for d in soup.find_all('div', class_="release_info"):
    info = {}
    info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
    info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
    info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
    info['期待度'] = ''
    info['劇情介紹'] = d.find('div', class_='release_text').find('span', class_='jq_text_overflow_180 jq_text_overflow_href_list').text.strip()

    if d.find('div', class_='leveltext'):    
        info['期待度'] = d.find('div', class_='leveltext').span.text
    infos.append(info)


res = []
for i, j in list(zip(infos, imgs)):
    res.append(dict(list(i.items()) + list(j.items())))
    
res




[]

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
 
 
options = Options()
options.add_argument("--disable-notifications")
 
chrome = webdriver.Chrome('./chromedriver', chrome_options=options)


 
time.sleep(1)
chrome.get("https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1")
chrome.get("https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2")



C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [18]:
print("helo")

helo


In [27]:
# Example： 奇摩電影
import requests
from bs4 import BeautifulSoup

url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
r = requests.get(url)
soup = BeautifulSoup(r.text)

infos = []
imgs = []
for d in soup.find_all('div', class_="release_info"):
    info = {}
    info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
    info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
    info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
    info['期待度'] = ''

    if d.find('div', class_='leveltext'):    
        info['期待度'] = d.find('div', class_='leveltext').span.text
    infos.append(info)
for d in soup.find_all('div', class_='release_foto'):
    img = {}
    img['封面照片網址'] = d.find('img')['src']
    imgs.append(img)

res = []
for i, j in list(zip(infos, imgs)):
    res.append(dict(list(i.items()) + list(j.items())))
    
res


[{'中文名稱': '玩命關頭9',
  '英文名稱': 'Fast & Furious 9',
  '上映日期': '2021-08-11',
  '期待度': '92%',
  '封面照片網址': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/July2021/tDAKOcY2jZvGxxZI3gcp-1024x1463.jpg'},
 {'中文名稱': '脫稿玩家',
  '英文名稱': 'Original title: Free Guy',
  '上映日期': '2021-08-12',
  '期待度': '100%',
  '封面照片網址': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/July2021/5rEaoBa3dspWWQS7Y0Rd-757x1080.jpg'},
 {'中文名稱': '被偷走的童年',
  '英文名稱': 'When Hitler Stole Pink Rabbit',
  '上映日期': '2021-08-13',
  '期待度': '98%',
  '封面照片網址': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/July2021/xh9896zFxerowai17WAb-756x1080.jpg'},
 {'中文名稱': '感動她77次',
  '英文名稱': '77 Heartwarmings',
  '上映日期': '2021-08-13',
  '期待度': '99%',
  '封面照片網址': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/July2021/eYXgH9CBAtBvzHqIgGQw-756x1080.jpg'},
 {'中文名稱': '汪汪隊立大功電影版',
  '英文名稱': 'PAW Patrol: The Movie',
  '上映日期': '2021-08-13',
  '期待度': '100%',
  '封面照片網址': 'https://movies.yahoo.com.tw/x/r/w420